<a href="https://colab.research.google.com/github/KSruthiVel/cookbook/blob/main/quickstarts/Caching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Context Caching Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Caching.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

This notebook introduces context caching with the Gemini API and provides examples of interacting with the Apollo 11 transcript using the Python SDK. For a more comprehensive look, check out [the caching guide](https://ai.google.dev/gemini-api/docs/caching?lang=python).

### Install dependencies

In [ ]:
%pip install -U -q "google-genai>=1.0.0"

In [ ]:
from google import genai
from google.genai import types

### Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [2]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

## Upload a file

A common pattern with the Gemini API is to ask a number of questions of the same document. Context caching is designed to assist with this case, and can be more efficient by avoiding the need to pass the same tokens through the model for each new request.

This example will be based on the transcript from the Apollo 11 mission.

Start by downloading that transcript.

In [3]:
!wget -q https://storage.googleapis.com/generativeai-downloads/data/a11.txt
!head a11.txt

INTRODUCTION

This is the transcription of the Technical Air-to-Ground Voice Transmission (GOSS NET 1) from the Apollo 11 mission.

Communicators in the text may be identified according to the following list.

Spacecraft:
CDR	Commander	Neil A. Armstrong
CMP	Command module pilot   	Michael Collins
LMP	Lunar module pilot	Edwin E. ALdrin, Jr.


Now upload the transcript using the [File API](../quickstarts/File_API.ipynb).

In [5]:
document = client.files.upload(file="a11.txt")

## Cache the prompt

Next create a [`CachedContent`](https://ai.google.dev/api/python/google/generativeai/protos/CachedContent) object specifying the prompt you want to use, including the file and other fields you wish to cache. In this example the [`system_instruction`](../quickstarts/System_instructions.ipynb) has been set, and the document was provided in the prompt.

In [6]:
# Note that caching requires a frozen model, e.g. one with a `-001` suffix.
model = "gemini-1.5-flash-002"
from google import genai
from google.genai import types
apollo_cache = client.caches.create(
      model=model,
      config={
          'contents': [document],
          'system_instruction': 'You are an expert at analyzing transcripts.',
      },
  )

apollo_cache

CachedContent(name='cachedContents/l5u17dyy5aoj', display_name='', model='models/gemini-1.5-flash-002', create_time=datetime.datetime(2025, 3, 17, 21, 12, 3, 889727, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 3, 17, 21, 12, 3, 889727, tzinfo=TzInfo(UTC)), expire_time=datetime.datetime(2025, 3, 17, 22, 12, 3, 248292, tzinfo=TzInfo(UTC)), usage_metadata=CachedContentUsageMetadata(audio_duration_seconds=None, image_count=None, text_count=None, total_token_count=323384, video_duration_seconds=None))

## Manage the cache expiry

Once you have a `CachedContent` object, you can update the expiry time to keep it alive while you need it.

In [7]:
from google.genai import types

client.caches.update(
    name=apollo_cache.name,
    config=types.UpdateCachedContentConfig(ttl="7200s")  # 2 hours in seconds
)
apollo_cache = client.caches.get(name=apollo_cache.name) # Get the updated cache
apollo_cache

CachedContent(name='cachedContents/l5u17dyy5aoj', display_name='', model='models/gemini-1.5-flash-002', create_time=datetime.datetime(2025, 3, 17, 21, 12, 3, 889727, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 3, 17, 21, 12, 16, 794318, tzinfo=TzInfo(UTC)), expire_time=datetime.datetime(2025, 3, 17, 23, 12, 16, 510549, tzinfo=TzInfo(UTC)), usage_metadata=CachedContentUsageMetadata(audio_duration_seconds=None, image_count=None, text_count=None, total_token_count=323384, video_duration_seconds=None))

## Use the cache for generation

As the `CachedContent` object refers to a specific model and parameters, you must create a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) using [`from_cached_content`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#from_cached_content). Then, generate content as you would with a directly instantiated model object.

In [8]:
response = client.models.generate_content(
    model=model,
    contents='Find a lighthearted moment from this transcript',
    config=types.GenerateContentConfig(
        cached_content=apollo_cache.name,
    )
)

print(response.text)

Several lighthearted moments punctuate the otherwise highly technical Apollo 11 transcript.  One of the clearest examples occurs on Tape 1/5, Page 5, between 00:00:05:35 and 00:00:05:39:


> CDR: You sure sound clear down there, Bruce. Sounds like you're sitting in your living room.
>
> CC: Oh, thank you. You all are coming through beautifully, too.

This friendly exchange shows the astronauts' relaxed demeanor despite the high-stakes nature of their mission.  The playful comparison of Mission Control to a living room highlights the strong rapport between the crew and CAPCOM.



You can inspect token usage through `usage_metadata`. Note that the cached prompt tokens are included in `prompt_token_count`, but excluded from the `total_token_count`.

In [9]:
response.usage_metadata

GenerateContentResponseUsageMetadata(cached_content_token_count=323384, candidates_token_count=148, prompt_token_count=323392, total_token_count=323540)

You can ask new questions of the model, and the cache is reused.

In [12]:
chat = client.chats.create(model=model, config={"cached_content": apollo_cache.name})

response = chat.send_message(message="Give me a quote from the most important part of the transcript.")
print(response.text)

The most important part of the transcript is the moment of the Moon landing.  The quote is:

**"Houston, Tranquility Base here. The Eagle has landed."**



In [11]:
response = chat.send_message(message="What was recounted after that?", config={"cached_content": apollo_cache.name})
print(response.text)


Following the announcement "The Eagle has landed," the transcript details the immediate post-landing activities and communications:

* **Acknowledgement and relief from Houston:** Mission Control expresses relief and congratulates the crew ("We copy you on the ground. You got a bunch of guys about to turn blue. We're breathing again. Thanks a lot.").

* **Post-landing procedures:**  The astronauts begin reporting on their status and performing post-landing checks, including system confirmations (Master ARM, engine stop, etc.) and venting of propellant.

* **Confirmation of landing status:** The astronauts confirm their safe landing and report on their immediate observations.

* **Communications with Columbia:**  The transcript shows communication with Michael Collins in the Command Module, confirming his status and relaying information.

* **Initial observations:** Armstrong describes the lunar surface as "fine and powdery," and discusses the lack of a significant crater from the desce

In [16]:
response.usage_metadata

GenerateContentResponseUsageMetadata(cached_content_token_count=323384, candidates_token_count=38, prompt_token_count=323397, total_token_count=323435)

As you can see, among the 323455 tokens, 323383 were cached (and thus less expensive) and only 236 were from the prompt.

Since the cached tokens are cheaper than the normal ones, it means this prompt was 75% cheaper that if you had not used caching. Check the [pricing here](https://ai.google.dev/pricing) for the up-to-date discount on cached tokens.

## Counting tokens

The `GenerativeModel` object can be used to count the tokens of a request in the same manner as a direct, uncached, model.

In [19]:
response = client.models.count_tokens(
    model=model,
    contents="How many people are involved in this transcript?",
)

response


CountTokensResponse(total_tokens=10, cached_content_token_count=None)

## Delete the cache

The cache has a small recurring storage cost (cf. [pricing](https://ai.google.dev/pricing)) so by default it is only saved for an hour. In this case you even set it up for a shorter amont of time (using `"ttl"`) of 2h.

Still, if you don't need you cache anymore, it is good practice to delete it proactively.

In [20]:
print(apollo_cache.name)
client.caches.delete(name=apollo_cache.name)

cachedContents/l5u17dyy5aoj


DeleteCachedContentResponse()

## Next Steps
### Useful API references:

If you want to know more about the caching API, you can check the full [API specifications](https://ai.google.dev/api/rest/v1beta/cachedContents) and the [caching documentation](https://ai.google.dev/gemini-api/docs/caching).

### Continue your discovery of the Gemini API

Check the File API notebook to know more about that API. The [vision capabilities](../quickstarts/Video.ipynb) of the Gemini API are a good reason to use the File API and the caching.
The Gemini API also has configurable [safety settings](../quickstarts/Safety.ipynb) that you might have to customize when dealing with big files.
